# Main Simulation Loop for Hourly Trading System

In [1]:
import re
import json
import pandas as pd
from openai import OpenAI
from query_model import query_model
from parse_response import parse_model_response
from update_portfolio import update_portfolio

# 1. Bullish

In [2]:
# === Load Data ===
PROMPT_PATH = "../prompts/wo_portfolio_prompts_dfs/bullish_prompts.csv"
PRICE_PATH = "./data/prices/all_prices_bullish.csv"

prompts_bullish = pd.read_csv(PROMPT_PATH, parse_dates=["timestamp"])
prices_bullish = pd.read_csv(PRICE_PATH, parse_dates=["timestamp"])

In [ ]:
prompts_bullish

In [ ]:
prices_bullish

## 1.1 Bullish 1st Period (2024-11-05 ~ 2024-11-11)

In [35]:
prompts_bullish_1 = prompts_bullish[:168]
prices_bullish_1 = prices_bullish[:168]

In [36]:
prompts_bullish_1

,timestamp,prompt
0,2024-11-05 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-11-05 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-11-05 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-11-05 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-11-05 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-11-11 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-11-11 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-11-11 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-11-11 22:00:00,You are an experienced multi-asset crypto trad...


In [37]:
prices_bullish_1

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-11-05 00:00:00,68006.8008,2400.2317,158.5760,0.164755,0.999386
1,2024-11-05 01:00:00,67969.5688,2409.7639,158.2116,0.162622,0.999366
2,2024-11-05 02:00:00,67857.8633,2403.9608,158.2175,0.161914,0.999453
3,2024-11-05 03:00:00,68216.7021,2419.2786,159.6270,0.162487,0.999443
4,2024-11-05 04:00:00,68308.7174,2427.8065,159.9727,0.162226,0.999453
...,...,...,...,...,...,...
163,2024-11-11 19:00:00,86248.7761,3306.3689,218.1010,0.317950,1.001894
164,2024-11-11 20:00:00,87004.3072,3356.0327,218.9628,0.321558,1.001616
165,2024-11-11 21:00:00,87986.4262,3327.2078,219.0813,0.330756,1.001156
166,2024-11-11 22:00:00,88053.3825,3344.7801,220.4168,0.327745,1.001057


In [116]:
import os
os.getcwd()

'/Users/zhangxingyue/Desktop/trading_agent/model'

In [ ]:
# === Tracking Structures ===
portfolio_history = []    # List of dicts to track portfolio history
portfolio_total_value_history = []    # List of total USD value per hour
scores_history = []    # List of dicts for model parsed scores
final_prompt_history = []    # List of actual prompts sent to the model
raw_response_history = []    # List of full LLM text outputs

# === Initial Portfolio ===
initial_portfolio = {
    "BTC": 0.0,
    "ETH": 0.0,
    "SOL": 0.0,
    "DOGE": 0.0,
    "USDT": 1_000_000.0    # Use underscore
}

In [ ]:
# === Simulation Loop ===
portfolio = initial_portfolio.copy()

for i in range(len(prompts_bullish_1)):              # TODO: TO CHANGE
    prompt_row = prompts_bullish_1.iloc[i]
    timestamp = prompt_row["timestamp"]
    prompt_base = prompt_row["prompt"]

    # 1. Replace "[PORTFOLIO HERE]" with current portfolio
    portfolio_text = "\n".join([f"- {token}: {round(amount, 4)}" for token, amount in portfolio.items()])
    prompt = prompt_base.replace("[PORTFOLIO HERE]", portfolio_text)
    final_prompt_history.append(prompt)

    # 2. Query the model
    response = query_model(prompt)
    raw_response_history.append(response)

    # 3. Parse the model response to get scores
    scores = parse_model_response(response)    # dict
    scores_history.append(scores.copy())

    # 4. Get price for the current timestamp
    price_row = prices_bullish_1.iloc[i]
    prices = {
        "BTC": price_row["btc_price"],
        "ETH": price_row["eth_price"],
        "SOL": price_row["sol_price"],
        "DOGE": price_row["doge_price"],
        "USDT": price_row["usdt_price"]
    }

    # 5. Update portfolio
    portfolio_updated = update_portfolio(
        current_portfolio=portfolio,
        scores=scores, 
        prices=prices,
        max_buy_ratio=0.007,    # PARAM
        sell_strength=0.5       # PARAM
    )
    portfolio_history.append(portfolio_updated.copy())

    # 6. Compute total value in USD
    total_value = sum(portfolio_updated[token] * prices[token] for token in portfolio_updated)
    portfolio_total_value_history.append(total_value)

    # 7. Move on to the next hour
    portfolio = portfolio_updated

In [ ]:
scores_history

In [ ]:
portfolio_history

In [ ]:
portfolio_total_value_history

In [ ]:
# === Save Results === 
results_bullish_1 = pd.DataFrame()
results_bullish_1["timestamp"] = prompts_bullish_1["timestamp"]
results_bullish_1["portfolio"] = portfolio_history
results_bullish_1["scores"] = scores_history
results_bullish_1["total_value"] = portfolio_total_value_history
results_bullish_1

In [ ]:
results_bullish_1.to_csv("data/results/num_bullish_2024-11-05_2024-11-11_0.007_0.5.csv", index=False)

In [ ]:
model_io_bullish_1 = pd.DataFrame()
model_io_bullish_1["timestamp"] = prompts_bullish_1["timestamp"]
model_io_bullish_1["prompt"] = final_prompt_history
model_io_bullish_1["response"] = raw_response_history
model_io_bullish_1

In [ ]:
model_io_bullish_1.to_csv("data/results/model_io_bullish_2024-11-05_2024-11-11_0.007_0.5.csv", index=False)

In [3]:
# Put everything in a function
def run_simulation(prompts_df, prices_df, max_buy_ratio: float, sell_strength: float):
    
    # === Tracking Structures ===
    portfolio_history = []    # List of dicts to track portfolio history
    portfolio_total_value_history = []    # List of total USD value per hour
    scores_history = []    # List of dicts for model parsed scores
    final_prompt_history = []    # List of actual prompts sent to the model
    raw_response_history = []    # List of full LLM text outputs

    # === Initial Portfolio ===
    initial_portfolio = {
        "BTC": 0.0,
        "ETH": 0.0,
        "SOL": 0.0,
        "DOGE": 0.0,
        "USDT": 1_000_000.0    # Use underscore
    }

    # === Simulation Loop ===
    portfolio = initial_portfolio.copy()

    for i in range(len(prompts_df)):              # TODO: TO CHANGE
        prompt_row = prompts_df.iloc[i]
        prompt_base = prompt_row["prompt"]

        # 1. Replace "[PORTFOLIO HERE]" with current portfolio
        portfolio_text = "\n".join([f"- {token}: {round(amount, 4)}" for token, amount in portfolio.items()])
        prompt = prompt_base.replace("[PORTFOLIO HERE]", portfolio_text)
        final_prompt_history.append(prompt)

        # 2. Query the model
        response = query_model(prompt)
        raw_response_history.append(response)

        # 3. Parse the model response to get scores
        scores = parse_model_response(response)    # dict
        scores_history.append(scores.copy())

        # 4. Get price for the current timestamp
        price_row = prices_df.iloc[i]
        prices = {
            "BTC": price_row["btc_price"],
            "ETH": price_row["eth_price"],
            "SOL": price_row["sol_price"],
            "DOGE": price_row["doge_price"],
            "USDT": price_row["usdt_price"]
        }

        # 5. Update portfolio
        portfolio_updated = update_portfolio(
            current_portfolio=portfolio,
            scores=scores, 
            prices=prices,
            max_buy_ratio=max_buy_ratio,    # PARAM
            sell_strength=sell_strength     # PARAM
        )
        portfolio_history.append(portfolio_updated.copy())

        # 6. Compute total value in USD
        total_value = sum(portfolio_updated[token] * prices[token] for token in portfolio_updated)
        portfolio_total_value_history.append(total_value)

        # 7. Move on to the next hour
        portfolio = portfolio_updated
        
    # === Save Results ===
    results = pd.DataFrame()
    results["timestamp"] = prompts_df["timestamp"]
    results["portfolio"] = portfolio_history
    results["scores"] = scores_history
    results["total_value"] = portfolio_total_value_history

    model_io = pd.DataFrame()
    model_io["timestamp"] = prompts_df["timestamp"]
    model_io["prompt"] = final_prompt_history
    model_io["response"] = raw_response_history
    
    return results, model_io

In [42]:
results_bullish_1, model_io_bullish_1 = run_simulation(prompts_df=prompts_bullish_1, prices_df=prices_bullish_1, max_buy_ratio=0.1, sell_strength=1.0)

In [43]:
results_bullish_1.to_csv("data/results/bullish1_0.1_1.0.csv", index=False)

In [44]:
model_io_bullish_1.to_csv("data/results/bullish1_model_io_0.1_1.0.csv", index=False)

## 1.2 Bullish 2nd Period (2024-11-18 ~ 2024-11-24)

In [9]:
prompts_bullish_2 = prompts_bullish[(prompts_bullish["timestamp"] >= "2024-11-18") & (prompts_bullish["timestamp"] < "2024-11-25")].copy().reset_index(drop=True)
prompts_bullish_2 

,timestamp,prompt
0,2024-11-18 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-11-18 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-11-18 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-11-18 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-11-18 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-11-24 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-11-24 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-11-24 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-11-24 22:00:00,You are an experienced multi-asset crypto trad...


In [10]:
print(prompts_bullish_2["prompt"][0])

You are an experienced multi-asset crypto trading agent operating in an hourly trading system. Your task is to make short-term asset allocation decisions across five cryptocurrencies: BTC, ETH, SOL, DOGE, and USDT. Your goal is to maximize your overall profit.

For each hour, you are provided with:

1. **Macroeconomic Context:** This hour's macro indicators such as interest rates, oil prices, the U.S. dollar index, gold, the S&P 500, and volatility.
2. **Recent Market Data**: A chronological snapshot of the last 4 hours of market data, including pricing, technical indicators, and on-chain activity for each asset (from oldest to newest).
3. **Daily News Summaries**: Important news summaries for each asset, published on the same day, from sources such as CoinDesk, Cointelegraph, or Decrypt.
4. **Current Portfolio Status**: Your current holdings for each asset, including the quantity of BTC, ETH, SOL, DOGE, and USDT.

## Objective:

Analyze both market data and news, based on your current

In [11]:
prices_bullish_2 = prices_bullish[(prices_bullish["timestamp"] >= "2024-11-18") & (prices_bullish["timestamp"] < "2024-11-25")].copy().reset_index(drop=True)
prices_bullish_2

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-11-18 00:00:00,89964.6550,3098.8883,235.2862,0.362572,1.000180
1,2024-11-18 01:00:00,90761.5745,3115.4375,237.6895,0.369684,1.000114
2,2024-11-18 02:00:00,90507.6656,3111.0859,239.7290,0.372299,1.000080
3,2024-11-18 03:00:00,90669.1319,3111.5463,244.1663,0.372189,1.000008
4,2024-11-18 04:00:00,90465.2619,3101.2543,242.5816,0.372905,1.000070
...,...,...,...,...,...,...
163,2024-11-24 19:00:00,96353.3222,3328.1993,248.6885,0.418103,1.000428
164,2024-11-24 20:00:00,96863.5395,3342.9295,250.8776,0.427531,1.000425
165,2024-11-24 21:00:00,97031.8654,3347.1320,251.1157,0.425843,1.000473
166,2024-11-24 22:00:00,98058.6001,3361.4731,253.4959,0.429371,1.000845


In [32]:
results_bullish_2, model_io_bullish_2 = run_simulation(prompts_df=prompts_bullish_2, prices_df=prices_bullish_2, max_buy_ratio=0.01, sell_strength=0.6)

In [33]:
results_bullish_2.to_csv("data/results/bullish2_0.01_0.6.csv", index=False)

In [34]:
model_io_bullish_2.to_csv("data/results/bullish2_model_io_0.01_0.6.csv", index=False)

## 1.3 Bullish 3nd Period (2024-11-26 ~ 2024-12-02)

In [17]:
prompts_bullish_3 = prompts_bullish[(prompts_bullish["timestamp"] >= "2024-11-26") & (prompts_bullish["timestamp"] < "2024-12-03")].copy().reset_index(drop=True)
prompts_bullish_3 

,timestamp,prompt
0,2024-11-26 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-11-26 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-11-26 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-11-26 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-11-26 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-12-02 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-12-02 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-12-02 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-12-02 22:00:00,You are an experienced multi-asset crypto trad...


In [18]:
print(prompts_bullish_2["prompt"][0])

You are an experienced multi-asset crypto trading agent operating in an hourly trading system. Your task is to make short-term asset allocation decisions across five cryptocurrencies: BTC, ETH, SOL, DOGE, and USDT. Your goal is to maximize your overall profit.

For each hour, you are provided with:

1. **Macroeconomic Context:** This hour's macro indicators such as interest rates, oil prices, the U.S. dollar index, gold, the S&P 500, and volatility.
2. **Recent Market Data**: A chronological snapshot of the last 4 hours of market data, including pricing, technical indicators, and on-chain activity for each asset (from oldest to newest).
3. **Daily News Summaries**: Important news summaries for each asset, published on the same day, from sources such as CoinDesk, Cointelegraph, or Decrypt.
4. **Current Portfolio Status**: Your current holdings for each asset, including the quantity of BTC, ETH, SOL, DOGE, and USDT.

## Objective:

Analyze both market data and news, based on your current

In [19]:
prices_bullish_3 = prices_bullish[(prices_bullish["timestamp"] >= "2024-11-26") & (prices_bullish["timestamp"] < "2024-12-03")].copy().reset_index(drop=True)
prices_bullish_3

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-11-26 00:00:00,93824.0263,3426.0400,235.3875,0.398912,0.999943
1,2024-11-26 01:00:00,94880.5905,3448.9250,237.5519,0.407070,1.000335
2,2024-11-26 02:00:00,94606.7521,3445.4416,238.7323,0.406764,1.000259
3,2024-11-26 03:00:00,94501.5469,3435.4769,237.5548,0.405551,1.000388
4,2024-11-26 04:00:00,94322.7893,3420.9553,237.1791,0.404918,1.000280
...,...,...,...,...,...,...
163,2024-12-02 19:00:00,95824.7216,3612.2056,223.8774,0.424218,1.000111
164,2024-12-02 20:00:00,95690.2023,3617.0690,222.9639,0.422470,1.000382
165,2024-12-02 21:00:00,95430.2323,3616.6530,222.2495,0.421125,1.000231
166,2024-12-02 22:00:00,95410.9242,3616.6072,224.4649,0.421820,1.000475


In [ ]:
prices_bullish_3

,timestamp,prompt
0,2024-11-26 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-11-26 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-11-26 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-11-26 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-11-26 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-12-02 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-12-02 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-12-02 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-12-02 22:00:00,You are an experienced multi-asset crypto trad...


In [112]:
results_bullish_3, model_io_bullish_3 = run_simulation(prompts_df=prompts_bullish_3, prices_df=prices_bullish_3, max_buy_ratio=0.01, sell_strength=1.0)

In [113]:
results_bullish_3.to_csv("data/results/num_0.01_1.0.csv", index=False)

In [114]:
model_io_bullish_3.to_csv("data/results/model_io_0.01_1.0.csv", index=False)

# 2. Bearish

In [51]:
# === Load Data ===
PROMPT_PATH = "../prompts/wo_portfolio_prompts_dfs/bearish_prompts.csv"
PRICE_PATH = "./data/prices/all_prices_bearish.csv"

prompts_bearish = pd.read_csv(PROMPT_PATH, parse_dates=["timestamp"])
prices_bearish = pd.read_csv(PRICE_PATH, parse_dates=["timestamp"])

In [52]:
prompts_bearish

,timestamp,prompt
0,2025-01-22 00:00:00,You are an experienced multi-asset crypto trad...
1,2025-01-22 01:00:00,You are an experienced multi-asset crypto trad...
2,2025-01-22 02:00:00,You are an experienced multi-asset crypto trad...
3,2025-01-22 03:00:00,You are an experienced multi-asset crypto trad...
4,2025-01-22 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
1123,2025-03-09 19:00:00,You are an experienced multi-asset crypto trad...
1124,2025-03-09 20:00:00,You are an experienced multi-asset crypto trad...
1125,2025-03-09 21:00:00,You are an experienced multi-asset crypto trad...
1126,2025-03-09 22:00:00,You are an experienced multi-asset crypto trad...


In [53]:
prices_bearish

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2025-01-22 00:00:00,105991.6920,3363.1587,253.0133,0.373459,0.999969
1,2025-01-22 01:00:00,105822.8884,3330.5308,252.4561,0.370568,0.999978
2,2025-01-22 02:00:00,105623.7806,3344.7382,252.6056,0.372697,0.999993
3,2025-01-22 03:00:00,105651.3822,3332.5598,254.4701,0.373087,1.000090
4,2025-01-22 04:00:00,105606.9847,3333.1036,255.9487,0.371775,0.999906
...,...,...,...,...,...,...
1123,2025-03-09 19:00:00,82671.2826,2041.4972,129.2483,0.171616,1.000643
1124,2025-03-09 20:00:00,83138.2554,2048.7784,128.4520,0.171554,1.000754
1125,2025-03-09 21:00:00,81939.4262,2026.2881,128.1068,0.169298,1.000761
1126,2025-03-09 22:00:00,80393.8015,2011.2138,127.0260,0.167321,1.000490


## 2.1 Bearish 1st Period (2025-01-30 ~ 2025-02-05)

In [54]:
prompts_bearish_1 = prompts_bearish[(prompts_bearish["timestamp"] >= "2025-01-30") & (prompts_bearish["timestamp"] < "2025-02-06")].copy().reset_index(drop=True)
prompts_bearish_1 

,timestamp,prompt
0,2025-01-30 00:00:00,You are an experienced multi-asset crypto trad...
1,2025-01-30 01:00:00,You are an experienced multi-asset crypto trad...
2,2025-01-30 02:00:00,You are an experienced multi-asset crypto trad...
3,2025-01-30 03:00:00,You are an experienced multi-asset crypto trad...
4,2025-01-30 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2025-02-05 19:00:00,You are an experienced multi-asset crypto trad...
164,2025-02-05 20:00:00,You are an experienced multi-asset crypto trad...
165,2025-02-05 21:00:00,You are an experienced multi-asset crypto trad...
166,2025-02-05 22:00:00,You are an experienced multi-asset crypto trad...


In [55]:
print(prompts_bearish_1["prompt"][0])

You are an experienced multi-asset crypto trading agent operating in an hourly trading system. Your task is to make short-term asset allocation decisions across five cryptocurrencies: BTC, ETH, SOL, DOGE, and USDT. Your goal is to maximize your overall profit.

For each hour, you are provided with:

1. **Macroeconomic Context:** This hour's macro indicators such as interest rates, oil prices, the U.S. dollar index, gold, the S&P 500, and volatility.
2. **Recent Market Data**: A chronological snapshot of the last 4 hours of market data, including pricing, technical indicators, and on-chain activity for each asset (from oldest to newest).
3. **Daily News Summaries**: Important news summaries for each asset, published on the same day, from sources such as CoinDesk, Cointelegraph, or Decrypt.
4. **Current Portfolio Status**: Your current holdings for each asset, including the quantity of BTC, ETH, SOL, DOGE, and USDT.

## Objective:

Analyze both market data and news, based on your current

In [56]:
prices_bearish_1 = prices_bearish[(prices_bearish["timestamp"] >= "2025-01-30") & (prices_bearish["timestamp"] < "2025-02-06")].copy().reset_index(drop=True)
prices_bearish_1

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2025-01-30 00:00:00,104034.4169,3124.6237,231.3156,0.326939,0.999980
1,2025-01-30 01:00:00,104268.1371,3138.0939,234.4911,0.329203,0.999967
2,2025-01-30 02:00:00,104664.5127,3154.5628,237.5322,0.331344,0.999944
3,2025-01-30 03:00:00,105212.6213,3200.8164,240.2198,0.333059,0.999958
4,2025-01-30 04:00:00,105488.4609,3197.5740,240.9110,0.334593,0.999981
...,...,...,...,...,...,...
163,2025-02-05 19:00:00,97393.6300,2775.8212,199.5984,0.259367,1.000148
164,2025-02-05 20:00:00,97333.7608,2763.2065,198.5431,0.257186,1.000073
165,2025-02-05 21:00:00,96936.5340,2787.1728,198.8923,0.257298,1.000103
166,2025-02-05 22:00:00,96489.6033,2760.1648,196.0353,0.254359,1.000093


In [62]:
results_bearish_1, model_io_bearish_1 = run_simulation(prompts_df=prompts_bearish_1, prices_df=prices_bearish_1, max_buy_ratio=0.01, sell_strength=0.6)

In [63]:
results_bearish_1.to_csv("data/results/num_0.01_0.6.csv", index=False)
model_io_bearish_1.to_csv("data/results/model_io_0.01_0.6.csv", index=False)

## 2.2 Bearish 2nd Period (2025-02-21 ~ 2025-02-27)

In [64]:
prompts_bearish_2 = prompts_bearish[(prompts_bearish["timestamp"] >= "2025-02-21") & (prompts_bearish["timestamp"] < "2025-02-28")].copy().reset_index(drop=True)
prompts_bearish_2 

,timestamp,prompt
0,2025-02-21 00:00:00,You are an experienced multi-asset crypto trad...
1,2025-02-21 01:00:00,You are an experienced multi-asset crypto trad...
2,2025-02-21 02:00:00,You are an experienced multi-asset crypto trad...
3,2025-02-21 03:00:00,You are an experienced multi-asset crypto trad...
4,2025-02-21 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2025-02-27 19:00:00,You are an experienced multi-asset crypto trad...
164,2025-02-27 20:00:00,You are an experienced multi-asset crypto trad...
165,2025-02-27 21:00:00,You are an experienced multi-asset crypto trad...
166,2025-02-27 22:00:00,You are an experienced multi-asset crypto trad...


In [65]:
print(prompts_bearish_2["prompt"][0])

You are an experienced multi-asset crypto trading agent operating in an hourly trading system. Your task is to make short-term asset allocation decisions across five cryptocurrencies: BTC, ETH, SOL, DOGE, and USDT. Your goal is to maximize your overall profit.

For each hour, you are provided with:

1. **Macroeconomic Context:** This hour's macro indicators such as interest rates, oil prices, the U.S. dollar index, gold, the S&P 500, and volatility.
2. **Recent Market Data**: A chronological snapshot of the last 4 hours of market data, including pricing, technical indicators, and on-chain activity for each asset (from oldest to newest).
3. **Daily News Summaries**: Important news summaries for each asset, published on the same day, from sources such as CoinDesk, Cointelegraph, or Decrypt.
4. **Current Portfolio Status**: Your current holdings for each asset, including the quantity of BTC, ETH, SOL, DOGE, and USDT.

## Objective:

Analyze both market data and news, based on your current

In [66]:
prices_bearish_2 = prices_bearish[(prices_bearish["timestamp"] >= "2025-02-21") & (prices_bearish["timestamp"] < "2025-02-28")].copy().reset_index(drop=True)
prices_bearish_2

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2025-02-21 00:00:00,98284.4827,2737.3206,174.6249,0.254370,1.000203
1,2025-02-21 01:00:00,98359.7411,2743.0331,174.4855,0.254056,1.000188
2,2025-02-21 02:00:00,98169.9476,2738.5563,173.5499,0.252975,1.000257
3,2025-02-21 03:00:00,98193.6412,2745.4751,173.4785,0.252717,1.000214
4,2025-02-21 04:00:00,98252.6086,2752.5385,174.2062,0.254021,1.000195
...,...,...,...,...,...,...
163,2025-02-27 19:00:00,84078.3636,2291.4460,136.2536,0.205129,0.999566
164,2025-02-27 20:00:00,83497.8537,2255.5817,136.6143,0.202992,0.999559
165,2025-02-27 21:00:00,84367.8004,2281.0867,137.3605,0.206229,0.999612
166,2025-02-27 22:00:00,84705.0385,2291.9031,138.0855,0.206745,0.999788


In [69]:
results_bearish_2, model_io_bearish_2 = run_simulation(prompts_df=prompts_bearish_2, prices_df=prices_bearish_2, max_buy_ratio=0.01, sell_strength=1.0)

In [70]:
results_bearish_2.to_csv("data/results/num_0.01_1.0.csv", index=False)
model_io_bearish_2.to_csv("data/results/model_io_0.01_1.0.csv", index=False)

## 2.3 Bearish 3nd Period (2025-03-03 ~ 2025-03-09)

In [71]:
prompts_bearish_3 = prompts_bearish[(prompts_bearish["timestamp"] >= "2025-03-03") & (prompts_bearish["timestamp"] < "2025-03-10")].copy().reset_index(drop=True)
prompts_bearish_3 

,timestamp,prompt
0,2025-03-03 00:00:00,You are an experienced multi-asset crypto trad...
1,2025-03-03 01:00:00,You are an experienced multi-asset crypto trad...
2,2025-03-03 02:00:00,You are an experienced multi-asset crypto trad...
3,2025-03-03 03:00:00,You are an experienced multi-asset crypto trad...
4,2025-03-03 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2025-03-09 19:00:00,You are an experienced multi-asset crypto trad...
164,2025-03-09 20:00:00,You are an experienced multi-asset crypto trad...
165,2025-03-09 21:00:00,You are an experienced multi-asset crypto trad...
166,2025-03-09 22:00:00,You are an experienced multi-asset crypto trad...


In [72]:
print(prompts_bearish_3["prompt"][0])

You are an experienced multi-asset crypto trading agent operating in an hourly trading system. Your task is to make short-term asset allocation decisions across five cryptocurrencies: BTC, ETH, SOL, DOGE, and USDT. Your goal is to maximize your overall profit.

For each hour, you are provided with:

1. **Macroeconomic Context:** This hour's macro indicators such as interest rates, oil prices, the U.S. dollar index, gold, the S&P 500, and volatility.
2. **Recent Market Data**: A chronological snapshot of the last 4 hours of market data, including pricing, technical indicators, and on-chain activity for each asset (from oldest to newest).
3. **Daily News Summaries**: Important news summaries for each asset, published on the same day, from sources such as CoinDesk, Cointelegraph, or Decrypt.
4. **Current Portfolio Status**: Your current holdings for each asset, including the quantity of BTC, ETH, SOL, DOGE, and USDT.

## Objective:

Analyze both market data and news, based on your current

In [73]:
prices_bearish_3 = prices_bearish[(prices_bearish["timestamp"] >= "2025-03-03") & (prices_bearish["timestamp"] < "2025-03-10")].copy().reset_index(drop=True)
prices_bearish_3

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2025-03-03 00:00:00,93384.2036,2467.1532,174.5393,0.235389,1.000213
1,2025-03-03 01:00:00,93130.9855,2460.2504,172.6851,0.231429,1.000096
2,2025-03-03 02:00:00,93361.6898,2448.5505,171.6575,0.229943,1.000003
3,2025-03-03 03:00:00,92763.2767,2442.0783,170.1082,0.228053,1.000001
4,2025-03-03 04:00:00,92911.7917,2451.3575,169.6771,0.229198,0.999997
...,...,...,...,...,...,...
163,2025-03-09 19:00:00,82671.2826,2041.4972,129.2483,0.171616,1.000643
164,2025-03-09 20:00:00,83138.2554,2048.7784,128.4520,0.171554,1.000754
165,2025-03-09 21:00:00,81939.4262,2026.2881,128.1068,0.169298,1.000761
166,2025-03-09 22:00:00,80393.8015,2011.2138,127.0260,0.167321,1.000490


In [78]:
results_bearish_3, model_io_bearish_3 = run_simulation(prompts_df=prompts_bearish_3, prices_df=prices_bearish_3, max_buy_ratio=0.01, sell_strength=1.0)

In [79]:
results_bearish_3.to_csv("data/results/num_0.01_1.0.csv", index=False)
model_io_bearish_3.to_csv("data/results/model_io_0.01_1.0.csv", index=False)

# 3. Sideways

In [80]:
# === Load Data ===
PROMPT_PATH = "../prompts/wo_portfolio_prompts_dfs/sideways_prompts.csv"
PRICE_PATH = "./data/prices/all_prices_sideways.csv"

prompts_sideways = pd.read_csv(PROMPT_PATH, parse_dates=["timestamp"])
prices_sideways = pd.read_csv(PRICE_PATH, parse_dates=["timestamp"])

In [81]:
prompts_sideways

,timestamp,prompt
0,2024-08-09 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-08-09 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-08-09 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-08-09 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-08-09 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
715,2024-09-07 19:00:00,You are an experienced multi-asset crypto trad...
716,2024-09-07 20:00:00,You are an experienced multi-asset crypto trad...
717,2024-09-07 21:00:00,You are an experienced multi-asset crypto trad...
718,2024-09-07 22:00:00,You are an experienced multi-asset crypto trad...


In [82]:
prices_sideways

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-08-09 00:00:00,61257.4370,2667.4673,161.4598,0.107046,1.000368
1,2024-08-09 01:00:00,61443.0406,2672.4434,160.5484,0.106111,1.000330
2,2024-08-09 02:00:00,61289.2862,2671.5673,158.6511,0.105677,1.000233
3,2024-08-09 03:00:00,61404.4036,2696.6841,159.2750,0.106241,1.000234
4,2024-08-09 04:00:00,60764.4714,2661.8933,156.3544,0.104452,1.000219
...,...,...,...,...,...,...
715,2024-09-07 19:00:00,54458.7186,2292.5390,128.5441,0.095121,1.000779
716,2024-09-07 20:00:00,54230.0669,2273.6253,127.5057,0.095193,1.000890
717,2024-09-07 21:00:00,53971.0896,2266.3987,126.9849,0.095151,1.000932
718,2024-09-07 22:00:00,54003.1837,2266.6692,126.9164,0.094951,1.000933


## 3.1 Sideways 1st Period (2024-08-09 ~ 2024-08-15)

In [83]:
prompts_sideways_1 = prompts_sideways[(prompts_sideways["timestamp"] < "2024-08-16")].copy().reset_index(drop=True)
prompts_sideways_1 

,timestamp,prompt
0,2024-08-09 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-08-09 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-08-09 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-08-09 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-08-09 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-08-15 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-08-15 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-08-15 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-08-15 22:00:00,You are an experienced multi-asset crypto trad...


In [85]:
print(prompts_sideways_1["prompt"][0])

You are an experienced multi-asset crypto trading agent operating in an hourly trading system. Your task is to make short-term asset allocation decisions across five cryptocurrencies: BTC, ETH, SOL, DOGE, and USDT. Your goal is to maximize your overall profit.

For each hour, you are provided with:

1. **Macroeconomic Context:** This hour's macro indicators such as interest rates, oil prices, the U.S. dollar index, gold, the S&P 500, and volatility.
2. **Recent Market Data**: A chronological snapshot of the last 4 hours of market data, including pricing, technical indicators, and on-chain activity for each asset (from oldest to newest).
3. **Daily News Summaries**: Important news summaries for each asset, published on the same day, from sources such as CoinDesk, Cointelegraph, or Decrypt.
4. **Current Portfolio Status**: Your current holdings for each asset, including the quantity of BTC, ETH, SOL, DOGE, and USDT.

## Objective:

Analyze both market data and news, based on your current

In [86]:
prices_sideways_1 = prices_sideways[(prices_sideways["timestamp"] < "2024-08-16")].copy().reset_index(drop=True)
prices_sideways_1

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-08-09 00:00:00,61257.4370,2667.4673,161.4598,0.107046,1.000368
1,2024-08-09 01:00:00,61443.0406,2672.4434,160.5484,0.106111,1.000330
2,2024-08-09 02:00:00,61289.2862,2671.5673,158.6511,0.105677,1.000233
3,2024-08-09 03:00:00,61404.4036,2696.6841,159.2750,0.106241,1.000234
4,2024-08-09 04:00:00,60764.4714,2661.8933,156.3544,0.104452,1.000219
...,...,...,...,...,...,...
163,2024-08-15 19:00:00,57091.1576,2551.4218,140.2620,0.099288,1.000282
164,2024-08-15 20:00:00,56650.4838,2546.8661,140.5492,0.099146,1.000402
165,2024-08-15 21:00:00,57658.0232,2573.6433,142.0542,0.099423,1.000345
166,2024-08-15 22:00:00,57663.0721,2578.3625,143.9796,0.100248,1.000339


In [91]:
results_sideways_1, model_io_sideways_1 = run_simulation(prompts_df=prompts_sideways_1, prices_df=prices_sideways_1, max_buy_ratio=0.005, sell_strength=0.8)

In [92]:
results_sideways_1.to_csv("data/results/num_0.005_0.8.csv", index=False)
model_io_sideways_1.to_csv("data/results/model_io_0.005_0.8.csv", index=False)

## 3.2 Sideways 2nd Period (2024-08-16 ~ 2024-08-22)

In [93]:
prompts_sideways_2 = prompts_sideways[(prompts_sideways["timestamp"] >= "2024-08-16") & (prompts_sideways["timestamp"] < "2024-08-23")].copy().reset_index(drop=True)
prompts_sideways_2 

,timestamp,prompt
0,2024-08-16 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-08-16 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-08-16 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-08-16 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-08-16 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-08-22 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-08-22 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-08-22 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-08-22 22:00:00,You are an experienced multi-asset crypto trad...


In [94]:
prices_sideways_2 = prices_sideways[(prices_sideways["timestamp"] >= "2024-08-16") & (prices_sideways["timestamp"] < "2024-08-23")].copy().reset_index(drop=True)
prices_sideways_2

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-08-16 00:00:00,57511.0192,2574.4121,142.1852,0.100528,1.000213
1,2024-08-16 01:00:00,57572.6957,2574.0516,142.2313,0.100279,1.000180
2,2024-08-16 02:00:00,58231.7486,2595.5974,143.4083,0.101136,1.000219
3,2024-08-16 03:00:00,57925.8048,2581.5499,142.2403,0.100729,1.000199
4,2024-08-16 04:00:00,58340.1159,2598.5421,142.7448,0.101548,1.000208
...,...,...,...,...,...,...
163,2024-08-22 19:00:00,60313.5790,2607.4105,142.8288,0.104863,1.000254
164,2024-08-22 20:00:00,60692.2651,2625.4600,143.4718,0.105351,1.000171
165,2024-08-22 21:00:00,60569.2689,2624.6693,143.4358,0.105405,1.000180
166,2024-08-22 22:00:00,60456.6558,2625.0155,143.2996,0.105489,1.000176


In [101]:
results_sideways_2, model_io_sideways_2 = run_simulation(prompts_df=prompts_sideways_2, prices_df=prices_sideways_2, max_buy_ratio=0.05, sell_strength=1.0)

In [102]:
results_sideways_2.to_csv("data/results/num_0.05_1.0.csv", index=False)
model_io_sideways_2.to_csv("data/results/model_io_0.05_1.0.csv", index=False)

## 3.3 Sideways 3rd Period (2024-08-28 ~ 2024-09-03)

In [103]:
prompts_sideways_3 = prompts_sideways[(prompts_sideways["timestamp"] >= "2024-08-28") & (prompts_sideways["timestamp"] < "2024-09-04")].copy().reset_index(drop=True)
prompts_sideways_3 

,timestamp,prompt
0,2024-08-28 00:00:00,You are an experienced multi-asset crypto trad...
1,2024-08-28 01:00:00,You are an experienced multi-asset crypto trad...
2,2024-08-28 02:00:00,You are an experienced multi-asset crypto trad...
3,2024-08-28 03:00:00,You are an experienced multi-asset crypto trad...
4,2024-08-28 04:00:00,You are an experienced multi-asset crypto trad...
...,...,...
163,2024-09-03 19:00:00,You are an experienced multi-asset crypto trad...
164,2024-09-03 20:00:00,You are an experienced multi-asset crypto trad...
165,2024-09-03 21:00:00,You are an experienced multi-asset crypto trad...
166,2024-09-03 22:00:00,You are an experienced multi-asset crypto trad...


In [104]:
prices_sideways_3 = prices_sideways[(prices_sideways["timestamp"] >= "2024-08-28") & (prices_sideways["timestamp"] < "2024-09-04")].copy().reset_index(drop=True)
prices_sideways_3

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2024-08-28 00:00:00,59125.7260,2442.3780,146.1594,0.098626,1.000273
1,2024-08-28 01:00:00,59409.9149,2444.8139,147.1026,0.099180,1.000176
2,2024-08-28 02:00:00,59380.1541,2461.7430,146.9407,0.098978,1.000364
3,2024-08-28 03:00:00,59669.6007,2480.4102,148.3523,0.099810,1.000350
4,2024-08-28 04:00:00,59368.0229,2473.0825,147.5815,0.099290,1.000390
...,...,...,...,...,...,...
163,2024-09-03 19:00:00,58047.7479,2447.7487,131.0297,0.097722,1.000005
164,2024-09-03 20:00:00,58236.9411,2463.8247,130.6697,0.098115,1.000039
165,2024-09-03 21:00:00,58143.6887,2465.3012,130.2371,0.097935,1.000168
166,2024-09-03 22:00:00,57889.9402,2454.4682,129.6336,0.097634,1.000189


In [109]:
results_sideways_3, model_io_sideways_3 = run_simulation(prompts_df=prompts_sideways_3, prices_df=prices_sideways_3, max_buy_ratio=0.1, sell_strength=1.0)

In [110]:
results_sideways_3.to_csv("data/results/num_0.1_1.0.csv", index=False)
model_io_sideways_3.to_csv("data/results/model_io_0.1_1.0.csv", index=False)

In [117]:
prices_bullish_1.to_csv("data/prices/prices_bullish_1.csv", index=False)
prices_bullish_2.to_csv("data/prices/prices_bullish_2.csv", index=False)
prices_bullish_3.to_csv("data/prices/prices_bullish_3.csv", index=False)

prices_bearish_1.to_csv("data/prices/prices_bearish_1.csv", index=False)
prices_bearish_2.to_csv("data/prices/prices_bearish_2.csv", index=False)
prices_bearish_3.to_csv("data/prices/prices_bearish_3.csv", index=False)

prices_sideways_1.to_csv("data/prices/prices_sideways_1.csv", index=False)
prices_sideways_2.to_csv("data/prices/prices_sideways_2.csv", index=False)
prices_sideways_3.to_csv("data/prices/prices_sideways_3.csv", index=False)

,timestamp,btc_price,eth_price,sol_price,doge_price,usdt_price
0,2025-03-03 00:00:00,93384.2036,2467.1532,174.5393,0.235389,1.000213
1,2025-03-03 01:00:00,93130.9855,2460.2504,172.6851,0.231429,1.000096
2,2025-03-03 02:00:00,93361.6898,2448.5505,171.6575,0.229943,1.000003
3,2025-03-03 03:00:00,92763.2767,2442.0783,170.1082,0.228053,1.000001
4,2025-03-03 04:00:00,92911.7917,2451.3575,169.6771,0.229198,0.999997
...,...,...,...,...,...,...
163,2025-03-09 19:00:00,82671.2826,2041.4972,129.2483,0.171616,1.000643
164,2025-03-09 20:00:00,83138.2554,2048.7784,128.4520,0.171554,1.000754
165,2025-03-09 21:00:00,81939.4262,2026.2881,128.1068,0.169298,1.000761
166,2025-03-09 22:00:00,80393.8015,2011.2138,127.0260,0.167321,1.000490
